# Includes & prerequisites

In [1]:
!cp /content/drive/MyDrive/Colab\ Notebooks/include/tuning_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/models.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/sca_utils.py /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/include/data_utils.py /content/
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.1 MB/s eta 0:00:00


In [2]:
from tuning_utils import CleanMemoryTuner, model_builder_synthetic_se_cnn, vgg_cbam_synthetic_builder
from models import cbam_block, vgg_cnn_cbam, cnn_squeeze_excite
from data_utils import SyntheticDataLoader

# Train the model with best hyperparameters on the full RPA dataset

In [3]:
model_config = {
    "CBAM": {
        "tuner_dir": "/content/drive/MyDrive/mbedtls_synthetic_traces/CBAM/hypertuning",
        "model_builder_fun": vgg_cbam_synthetic_builder,
        "model_fun": vgg_cnn_cbam
    },
    "SE": {
        "tuner_dir": "/content/drive/MyDrive/mbedtls_synthetic_traces/hyperparameter_tuning_efficient",
        "model_builder_fun": model_builder_synthetic_se_cnn,
        "model_fun": cnn_squeeze_excite
    }
}

In [4]:
# ----- Configuration -----

MODEL = "CBAM"
BYTE = 15

H5_FILE_PATH = f"/content/drive/MyDrive/mbedtls_synthetic_traces/RPA_traces/v2_profiling_windows/byte_{BYTE}.h5"

RUN_ROOT = "/content/drive/MyDrive/mbedtls_synthetic_traces"
BATCH_SIZE = 128
EPOCHS = 60
VALIDATION_SPLIT = 0.2
# LR = 1e-3
SEED = 1917

In [5]:
# ----- Reproducibility -----

import os, json, random, numpy as np, tensorflow as tf
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [6]:
# ----- Paths

from datetime import datetime
run_id = datetime.now().strftime(f"B{BYTE}_{MODEL}_%Y%m%d_%H%M%S")

RUN_DIR = f"{RUN_ROOT}/runs_{MODEL}{run_id}"
CKPT_BEST = f"{RUN_DIR}/best.keras"
LOG_JSON = f"{RUN_DIR}/config.json"
os.makedirs(RUN_DIR, exist_ok=True)

## Load the training data

Based on the `H5_FILE_PATH`, load the appropriate training data (in this case the RPA dataset).

In [7]:
h5_loader = SyntheticDataLoader(file_path=H5_FILE_PATH)
ds_train, ds_full_val, INPUT_DIM, train_size, val_size = h5_loader.create_dataset(
    window_index=0,
    batch_size=BATCH_SIZE,
    val_split=VALIDATION_SPLIT
)

Dataset created from 'byte_15.h5':
  - Total samples: 6001
  - Training samples: 4801
  - Validation samples: 1200


In [8]:
ds_val_for_fit = ds_full_val.map(lambda trace, label, pt, key: (trace, label))

## Load the model from tuning directory

Get the best hyperparameters from the tuning runs on a random dataset and rebuild the model.

In [9]:
tuner = CleanMemoryTuner(
    hypermodel=lambda hp: model_config[MODEL]['model_builder_fun'](
            hp,
            model_fun=model_config[MODEL]['model_fun'],
            input_dim=INPUT_DIM
        ),
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    directory=model_config[MODEL]['tuner_dir'],
    project_name=f"tuning_byte_{BYTE}",
    overwrite=False
)

Reloading Tuner from /content/drive/MyDrive/mbedtls_synthetic_traces/CBAM/hypertuning/tuning_byte_15/tuner0.json


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"best_hps: {best_hps.values}")
model = tuner.hypermodel.build(best_hps)

best_hps: {'filter_scale': 1.5, 'dropout': 0.0, 'cbam_ratio': 8, 'batch_normalize': True, 'optimizer': 'rmsprop', 'learning_rate': 5e-05}


In [11]:
model.summary()

Model: "cbam_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 701, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 701, 96)   │      1,152 │ input_layer[0][0] │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 701, 96)   │        384 │ block1_conv1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool1        │ (None, 350, 96)   │          0 │ batch_normalizat… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 350, 192)  │    202,944 │ block1_pool1[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 350, 192)  │        768 │ block2_conv1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 192)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 192)       │          0 │ batch_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 24)        │      4,632 │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 192)       │      4,800 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 192)       │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_1[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 192)       │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 192)    │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 350, 192)  │          0 │ batch_normalizat… │
│                     │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ channelwise_pool    │ (None, 350, 2)    │          0 │ multiply[0][0]    │
│ (ChannelwisePool)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 350, 1)    │         14 │ channelwise_pool… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 350, 192)  │          0 │ multiply[0][0],   │
│ (Multiply)          │                   │            │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 94,810,260 (361.67 MB)

 Trainable params: 94,805,844 (361.66 MB)

 Non-trainable params: 4,416 (17.25 KB)

In [12]:
print(model.optimizer.learning_rate)

<Variable path=rmsprop/learning_rate, shape=(), dtype=float32, value=4.999999873689376e-05>


## Run the training

In [13]:
csv_log = tf.keras.callbacks.CSVLogger(f"{RUN_DIR}/train_log.csv")
cbs = [
    tf.keras.callbacks.ModelCheckpoint(CKPT_BEST, monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True),
    csv_log
]

In [14]:
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = val_size // BATCH_SIZE

print("\n--- Starting Model Training on HDF5 Data ---")
history = model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_val_for_fit,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=cbs,
    verbose=2
)


--- Starting Model Training on HDF5 Data ---
Epoch 1/60
37/37 - 22s - 585ms/step - accuracy: 0.0051 - loss: 5.5451 - val_accuracy: 0.0052 - val_loss: 5.5449
Epoch 2/60


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


37/37 - 6s - 164ms/step - accuracy: 0.0000e+00 - loss: 5.5415 - val_accuracy: 0.0061 - val_loss: 5.5437
Epoch 3/60
37/37 - 12s - 313ms/step - accuracy: 0.0091 - loss: 5.5349 - val_accuracy: 0.0078 - val_loss: 5.5414
Epoch 4/60
37/37 - 2s - 67ms/step - accuracy: 0.0154 - loss: 5.5355 - val_accuracy: 0.0052 - val_loss: 5.5422
Epoch 5/60
37/37 - 11s - 309ms/step - accuracy: 0.0087 - loss: 5.5206 - val_accuracy: 0.0052 - val_loss: 5.5378
Epoch 6/60
37/37 - 6s - 160ms/step - accuracy: 0.0154 - loss: 5.5067 - val_accuracy: 0.0069 - val_loss: 5.5370
Epoch 7/60
37/37 - 13s - 339ms/step - accuracy: 0.0082 - loss: 5.4985 - val_accuracy: 0.0043 - val_loss: 5.5344
Epoch 8/60
37/37 - 6s - 159ms/step - accuracy: 0.0000e+00 - loss: 5.4981 - val_accuracy: 0.0069 - val_loss: 5.5322
Epoch 9/60
37/37 - 8s - 219ms/step - accuracy: 0.0154 - loss: 5.4639 - val_accuracy: 0.0052 - val_loss: 5.5339
Epoch 10/60
37/37 - 7s - 192ms/step - accuracy: 0.0000e+00 - loss: 5.5106 - val_accuracy: 0.0069 - val_loss: 5.53

In [15]:
model.save(f"{RUN_DIR}/final.keras")

In [16]:
print([l.numpy() for _, l in ds_train.unbatch().batch(10)])

[array([235, 149,  88, 173, 162,  60, 131, 149, 181,  39], dtype=int32), array([ 27, 167, 153, 208, 198,  84,  53, 237,  14,  47], dtype=int32), array([ 58, 239,  48, 155,  66,  39,   4, 242, 206,  61], dtype=int32), array([201,  59,  81, 234, 115,  31,  42,  66,  10,  33], dtype=int32), array([187,  26, 190, 158, 214,  60, 232,  82, 206,   5], dtype=int32), array([ 95, 153, 127,  77, 167,  64, 194, 235,  75, 247], dtype=int32), array([218,  38,  31,  14,  82, 109,  41, 201, 178, 176], dtype=int32), array([  9,  35, 205, 114, 112, 158,  66,  65, 192, 191], dtype=int32), array([ 29,  95, 124, 150, 169, 126,  12, 225,  83,  57], dtype=int32), array([134, 227, 104,  50, 165, 221, 137, 116, 213, 208], dtype=int32), array([149, 245, 187, 128, 125,  73, 220, 147, 110,  97], dtype=int32), array([241,  25, 142,  28,  22,  17,  34,   2, 179, 123], dtype=int32), array([146, 172, 175, 174, 167,  74, 250, 100, 162, 232], dtype=int32), array([141,  46, 245,  52, 209, 225,  14, 242, 140, 189], dtype

In [17]:
ds_val_X = ds_full_val.map(lambda trace,label,pt,key: trace)
raw = model.predict(ds_val_X, verbose=0)

FROM_LOGITS = True

if FROM_LOGITS:
  val_logits = raw
else:
  val_logits = tf.math.log(tf.clip_by_value(raw, 1e-12, 1.0)).numpy()

val_labels = np.array([int(lbl.numpy()) for _, lbl, _, _ in ds_full_val.unbatch()], dtype=np.int64)

np.savez(f"{RUN_DIR}/val_logits_labels.npz", logits=val_logits, labels=val_labels)